Install Lib

In [ ]:
%%capture
!pip install torch torchvision timm

In [ ]:
# Import necessary libraries
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.models as models
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import os
import wandb

Unzip dataset

In [ ]:
%%capture
%%bash
unzip all.zip -d datasetss/

init wanb for logging

In [ ]:
wandb.init(project="resnet101-training-project", name="all-4")

Define dataset paths (Update this to match your folder structure)

In [29]:
# Define dataset paths (Update this to match your folder structure)
data_dir = '/content/datasetss'  # Update with your dataset path
train_dir = os.path.join(data_dir, 'train')
val_dir = os.path.join(data_dir, 'valid')

Define transformations for data augmentation

In [30]:
# Define transformations for data augmentation
transform = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(10),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

Load datasets

In [31]:
# Load datasets
train_dataset = ImageFolder(train_dir, transform=transform['train'])
val_dataset = ImageFolder(val_dir, transform=transform['val'])

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)



Load pre-trained ResNet101 model

In [32]:
# Load pre-trained ResNet101 model
model = models.resnet101(pretrained=True)
num_classes = len(train_dataset.classes)  # Get number of classes from dataset

# Modify the final layer to match our number of classes
model.fc = nn.Sequential(
    nn.Dropout(0.3),  # Dropout Layer added (30% dropout)
    nn.Linear(model.fc.in_features, num_classes)
)


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet101_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


# Define loss function and optimizer

In [33]:
# Set device (GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001, weight_decay=0.0001)


# Training loop with best model saving

In [34]:
# Training loop with best model saving
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=5):
    best_val_acc = 0.0
    best_model_path = '/content/fruit.pth'

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

        train_loss = running_loss / len(train_loader)
        train_acc = 100 * correct / total

        # Validation
        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0

        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, predicted = outputs.max(1)
                total += labels.size(0)
                correct += predicted.eq(labels).sum().item()

        val_loss = val_loss / len(val_loader)
        val_acc = 100 * correct / total

        print(f"Epoch {epoch+1}/{num_epochs} | Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}% | Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.2f}%")
        wandb.log({
            "epoch": epoch+1,
            "train_loss": train_loss,
            "train_accuracy": train_acc,
            "val_loss": val_loss,
            "val_accuracy": val_acc
        })
        # Save the best model
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), best_model_path)
            print(f"Best model saved with accuracy: {best_val_acc:.2f}%")
            torch.save(model.state_dict(), best_model_path)
    print("Training complete!")
    return model

Train model

In [35]:
trained_model = train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=1)

print(f'Best model saved at /content/fruit.pth')

Epoch 1/1 | Train Loss: 0.0955 | Train Acc: 97.00% | Val Loss: 0.0653 | Val Acc: 98.27%
Best model saved with accuracy: 98.27%
Training complete!
Best model saved at /content/fruit.pth


HOW TO CALL MY MODEL

Load trained model and class

In [79]:
import os
import torch
import torchvision.transforms as transforms
from PIL import Image
import torchvision.models as models

# Load class names (make sure they match your dataset)
class_names = ["good", "rotten"]  # Update with your actual class names

# Load trained model
num_classes = len(class_names)
model = models.resnet101(pretrained=False)
model.fc = torch.nn.Linear(model.fc.in_features, num_classes)
model.load_state_dict(torch.load('/content/fruit.pth', map_location=torch.device('cpu')))
model.eval()  # Set to evaluation mode

print("Model Loaded Successfully!")

<ipython-input-79-ed3d3772604e>:14: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('/content/fruit.pth', map_location=torch.device('cpu')))


Model Loaded Successfully!


Define image transformation (Same as training)

In [80]:


# Define image transformation (Same as training)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])


Predict the class and confidence score for an image.

In [81]:
def predict(image_path):
    """Predict the class and confidence score for an image."""
    image = Image.open(image_path)  # Load image
    image = transform(image).unsqueeze(0)  # Apply transformations and add batch dimension

    with torch.no_grad():
        outputs = model(image)  # Forward pass
        probabilities = torch.nn.functional.softmax(outputs, dim=1)  # Convert to probabilities
        confidence, predicted_class = torch.max(probabilities, 1)  # Get top class and confidence

    predicted_label = class_names[predicted_class.item()]  # Get class name
    confidence_percentage = confidence.item() * 100  # Convert confidence to percentage

    return predicted_label, confidence_percentage

call the function to predict the image from a folder

In [82]:
image_folder = "/content/dataset/starfruit"  # Change to your folder path
output_results = []

for filename in os.listdir(image_folder):
    if filename.endswith((".jpg", ".png", ".jpeg")):  # Ensure it's an image file
        image_path = os.path.join(image_folder, filename)
        predicted_label, confidence_percentage = predict(image_path)
        output_results.append((filename, predicted_label, confidence_percentage))

        print(f"Image: {filename}")
        print(f"Predicted Class: {predicted_label}")
        print(f"Confidence Score: {confidence_percentage:.2f}%")
        print("-" * 30)

# Optionally save results to a file
output_file = "/content/prediction_results.txt"
with open(output_file, "w") as f:
    for filename, label, confidence in output_results:
        f.write(f"{filename}: {label} ({confidence:.2f}%)\n")

print(f"Predictions saved to {output_file}")

Image: starfruit_1739689948669.jpg
Predicted Class: good
Confidence Score: 99.48%
------------------------------
Image: starfruit_1739689946826.jpg
Predicted Class: good
Confidence Score: 99.74%
------------------------------
Image: starfruit_1739689945084.jpg
Predicted Class: good
Confidence Score: 99.76%
------------------------------
Image: starfruit_1739689946445.jpg
Predicted Class: good
Confidence Score: 99.30%
------------------------------
Image: starfruit_1739689947264.jpg
Predicted Class: good
Confidence Score: 99.91%
------------------------------
Image: starfruit_1739689943731.jpg
Predicted Class: good
Confidence Score: 99.23%
------------------------------
Image: starfruit_1739689945874.jpg
Predicted Class: good
Confidence Score: 99.57%
------------------------------
Image: starfruit_1739689946447.jpg
Predicted Class: good
Confidence Score: 99.92%
------------------------------
Image: starfruit_1739689946635.jpg
Predicted Class: good
Confidence Score: 99.28%
--------------

In [23]:
%%capture
%%bash
unzip bad_fruits_roi.zip -d datasets/

In [22]:
%%capture
%%bash
unzip good_fruits_roi.zip -d dataset/

Classification Report

In [83]:
import os
import torch
from PIL import Image
from sklearn.metrics import classification_report

# Ensure model is on the correct device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()  # Set to evaluation mode

# Define image folder (where images are stored)
image_folder = "/content/datasetss/valid"  # Adjust this path

# Manually define class names
class_names = ["good", "rotten"]  # Ensure these match your dataset structure
class_to_idx = {cls_name: idx for idx, cls_name in enumerate(class_names)}
print(class_to_idx)
# Define loss function (same as training)
criterion = torch.nn.CrossEntropyLoss()

# Store true and predicted labels
true_labels = []
predicted_labels = []
total_loss = 0.0
correct_predictions = 0
total_images = 0

def predict(image_path, true_label_idx):
    """Predict the class and confidence score for an image while computing loss."""
    image = Image.open(image_path)
    image = transform(image).unsqueeze(0).to(device)  # Move image to same device as model

    with torch.no_grad():
        outputs = model(image)  # Model is on the same device
        probabilities = torch.nn.functional.softmax(outputs, dim=1)  # Convert to probabilities
        confidence, predicted_class = torch.max(probabilities, 1)
        print(true_label_idx)
        # Move true label to the same device before computing loss
        true_label_tensor = torch.tensor([true_label_idx], device=device)
        loss = criterion(outputs, true_label_tensor)

    predicted_label = class_names[predicted_class.item()]
    confidence_percentage = confidence.item() * 100
    return predicted_label, confidence_percentage, loss.item(), predicted_class.item()

# Process all images
output_results = []

for class_folder in class_names:  # Iterate over predefined class names
    class_folder_path = os.path.join(image_folder, class_folder)
    if not os.path.isdir(class_folder_path):  # Skip if not a valid directory
        print(f"Warning: Folder not found - {class_folder_path}")
        continue

    for filename in os.listdir(class_folder_path):
        if filename.endswith((".jpg", ".png", ".jpeg")):
            image_path = os.path.join(class_folder_path, filename)

            # Predict label and compute loss
            predicted_label, confidence_percentage, loss, predicted_idx = predict(image_path, class_to_idx[class_folder])

            # Store true and predicted labels
            true_labels.append(class_to_idx[class_folder])  # Store true labels as indices
            predicted_labels.append(class_to_idx[predicted_label])

            # Update test loss and accuracy calculations
            total_loss += loss
            total_images += 1
            if predicted_idx == class_to_idx[class_folder]:  # Check if prediction is correct
                correct_predictions += 1

            output_results.append((filename, class_folder, predicted_label, confidence_percentage, loss))

            print(f"Image: {filename} | True: {class_folder} | Predicted: {predicted_label} | Confidence: {confidence_percentage:.2f}% | Loss: {loss:.4f}")

# Check if true_labels and predicted_labels are populated
if not true_labels or not predicted_labels:
    raise ValueError("No labels were detected. Ensure images are inside 'good/' and 'rotten/' subfolders.")

# Compute Test Loss and Test Accuracy
test_loss = total_loss / total_images
test_accuracy = (correct_predictions / total_images) * 100

# Save results to file
output_file = "/content/prediction_results.txt"
with open(output_file, "w") as f:
    for filename, true_label, predicted_label, confidence, loss in output_results:
        f.write(f"{filename}: True: {true_label}, Predicted: {predicted_label} ({confidence:.2f}%) | Loss: {loss:.4f}\n")

print(f"Predictions saved to {output_file}")

# Generate and print classification report
print("True Labels:", true_labels)
print("Predicted Labels:", predicted_labels)
print("Class Names Mapping:", class_to_idx)
report = classification_report(true_labels, predicted_labels, target_names=class_names)

print("\nClassification Report:")
print(report)
print(f"\nTest Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.2f}%")


{'good': 0, 'rotten': 1}
0
Image: pear_1739689903499_jpg.rf.b685ec5423d93d08a7c83d440343afd7.jpg | True: good | Predicted: good | Confidence: 99.98% | Loss: 0.0002
0
Image: pear_1739689905834_jpg.rf.017f9e037970f1ff157cf5e2e5a5b47a.jpg | True: good | Predicted: good | Confidence: 99.32% | Loss: 0.0068
0
Image: pear_1739689903073_jpg.rf.68de94ab205e55c0497b7f3d082b779f.jpg | True: good | Predicted: good | Confidence: 99.80% | Loss: 0.0020
0
Image: apple_1739689875759_jpg.rf.8ad9d8eafe8ce16eda2a9a39b2ef3b90.jpg | True: good | Predicted: good | Confidence: 90.84% | Loss: 0.0961
0
Image: banana_1739689929969_jpg.rf.426a8dbaa169e9ae46ca53bcb476288c.jpg | True: good | Predicted: good | Confidence: 99.95% | Loss: 0.0005
0
Image: banana_1739689929536_jpg.rf.c20a7efca61893e41b2538f169a287b1.jpg | True: good | Predicted: good | Confidence: 99.98% | Loss: 0.0002
0
Image: kiwi_1739689838537_jpg.rf.81dfecf6e8207697262dcbeb11510f06.jpg | True: good | Predicted: good | Confidence: 99.90% | Loss: 0.00